<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 2 ( Recuperación de Información) 

## Profesora: Lisibonny Beato
### Período 3-2023-2024</div>


In [21]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from os import path

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>El propósito de esta asignación es que el estudiante ponga en práctica la construcción de sistemas de recuperación de información basado en el modelo clásico de RI Vector Space Model, así como también que evalue la efectividad de estos modelos mediante el uso de una colección de referencia (Benchmark).
<br />
<br />
Para esta práctica estará utilizando el siguiente repositorio:https://github.com/oussbenk/cranfield-trec-dataset. En el mismo encontrarán los archivos  cran.all.1400.xml, cran.qry.xml, cranqrel.trec.txt:
<ul>
<li>cran.all.1400.xml contiene 1,400 resumenes de artículos científicos.</li>
<li>cran.qry.xml 225 términos que representan consultas.</li>
<li>cranqrel.trec.txt contiene los juicios de relevancia a dichas consultas.</li>
    </ul>  
<br />
<br />
Estudie en detalle la estructura y el contenido de este conjunto de documentos provistos antes de comenzar.    
<br />
<br />
En este trabajo, aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Ejercicio 1
### Puntuación máxima de la tarea: 3 puntos
#### Limpieza y preparación de los datos, utilizando distintas técnicas de las ya vistas en clases. Para esta tarea utilizará el archivo cran.all.1400.xml, específicamente sus columnas title y text.


In [22]:
import xml.etree.ElementTree as ElementTree

with open('cranfield-trec-dataset/cran.all.1400.xml', 'r') as f:  # Reading file
    xml = f.read()

xml = '<root>' + xml + '</root>'  # Let's add a root tag

root = ElementTree.fromstring(xml)

# Simple loop through each document
df = pd.DataFrame(columns=['docno', 'title', 'author', 'bib', 'text'])

for doc in root:
    docno = doc.find('docno').text.strip() if doc.find(
        'docno').text is not None else ''
    title = doc.find('title').text.strip() if doc.find(
        'title').text is not None else ''
    author = doc.find('author').text.strip() if doc.find(
        'author').text is not None else ''
    bib = doc.find('bib').text.strip() if doc.find(
        'bib').text is not None else ''
    text = doc.find('text').text.strip() if doc.find(
        'text').text is not None else ''

    new_row = pd.DataFrame({
        'docno': [docno],
        'title': [title],
        'author': [author],
        'bib': [bib],
        'text': [text]
    })
    df = pd.concat([df, new_row], ignore_index=True)

df

,docno,title,author,bib,text
0,1,experimental investigation of the aerodynamics...,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...
1,2,simple shear flow past a flat plate in an inco...,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...
2,3,the boundary layer in simple shear flow past a...,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...
3,4,approximate solutions of the incompressible la...,"yen,k.t.","j. ae. scs. 22, 1955, 728.",approximate solutions of the incompressible la...
4,5,one-dimensional transient heat conduction into...,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...
...,...,...,...,...,...
1395,1396,shear buckling of clamped and simply-supported...,"cook,i.t. and rockey,k.c.","aero. quart. 13, 1962, 41.",shear buckling of clamped and simply-supported...
1396,1397,critical shear stress of an infinitely long si...,"stein,m. and fralich,r.w.","naca tn.1851, 1949.",critical shear stress of an infinitely long si...
1397,1398,stability of rectangular plates under shear an...,"way,s.","j. app. mech. 3, 1936, a131.",stability of rectangular plates under shear an...
1398,1399,buckling of transverse stiffened plates under ...,"wang,t.k.","j.app.mech. 3,1947, a269.",buckling of transverse stiffened plates under ...


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

In [26]:
import spacy

nlp = spacy.load("en_core_web_sm")

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
  El objetivo de ese metodo data_cleaning es limpiar y preparar los datos del DataFrame df. para normalizar el texto y evitar problemas de incompatibilidad co respecto a la consulta. primero se convierte el texto a minúsculas, luego se eliminan los signos de puntuación y caracteres especiales, por ejemplo, si el texto contiene hello y el query contiene "!Hello!" independientemente de los signos el texto es relevante para la consulta y esta limpieza hará que sea más facil la obtención de estos textos, se tokeniza el texto en palabras individuales, se eliminan las stopwords (palabras comunes que no aportan significado) y finalmente se aplica lematización para reducir las palabras a su forma base. El resultado es un DataFrame con columnas 'title' y 'text' que contienen listas de palabras limpias y normalizadas.
</div>

In [24]:
def data_cleaning(dataframe):
    """
    Limpia y prepara los datos del DataFrame para normalizar el texto.
    Args:
        dataframe (pd.DataFrame): DataFrame que contiene las columnas 'title' y 'text'.
    Returns:
        pd.DataFrame: DataFrame con las columnas 'title' y 'text' limpias y normalizadas.
    """

    # Convert to lowercase
    dataframe['title'] = dataframe['title'].str.lower()
    dataframe['text'] = dataframe['text'].str.lower()

    # Remove punctuation and special characters
    dataframe['title'] = dataframe['title'].apply(
        lambda x: re.sub(r'[^\w\s]', '', x))
    dataframe['text'] = dataframe['text'].apply(
        lambda x: re.sub(r'[^\w\s]', '', x))

    # Tokenization
    dataframe['title'] = dataframe['title'].apply(word_tokenize)
    dataframe['text'] = dataframe['text'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    dataframe['title'] = dataframe['title'].apply(
        lambda x: [word for word in x if word not in stop_words])
    dataframe['text'] = dataframe['text'].apply(
        lambda x: [word for word in x if word not in stop_words])

    #Lemmatization
    dataframe['title'] = dataframe['title'].apply(
        lambda x: [token.lemma_ for token in nlp(' '.join(x))])
    dataframe['text'] = dataframe['text'].apply(
        lambda x: [token.lemma_ for token in nlp(' '.join(x))])

    return dataframe

In [25]:
df = data_cleaning(df)
df

,docno,title,author,bib,text
0,1,"[experimental, investigation, aerodynamic, win...","brenckman,m.","j. ae. scs. 25, 1958, 324.","[experimental, investigation, aerodynamic, win..."
1,2,"[simple, shear, flow, past, flat, plate, incom...",ting-yili,"department of aeronautical engineering, rensse...","[simple, shear, flow, past, flat, plate, incom..."
2,3,"[boundary, layer, simple, shear, flow, past, f...",m. b. glauert,"department of mathematics, university of manch...","[boundary, layer, simple, shear, flow, past, f..."
3,4,"[approximate, solution, incompressible, lamina...","yen,k.t.","j. ae. scs. 22, 1955, 728.","[approximate, solution, incompressible, lamina..."
4,5,"[onedimensional, transient, heat, conduction, ...","wasserman,b.","j. ae. scs. 24, 1957, 924.","[onedimensional, transient, heat, conduction, ..."
...,...,...,...,...,...
1395,1396,"[shear, buckle, clamp, simplysupporte, infinit...","cook,i.t. and rockey,k.c.","aero. quart. 13, 1962, 41.","[shear, buckle, clamp, simplysupporte, infinit..."
1396,1397,"[critical, shear, stress, infinitely, long, si...","stein,m. and fralich,r.w.","naca tn.1851, 1949.","[critical, shear, stress, infinitely, long, si..."
1397,1398,"[stability, rectangular, plate, shear, bend, f...","way,s.","j. app. mech. 3, 1936, a131.","[stability, rectangular, plate, shear, bend, f..."
1398,1399,"[buckle, transverse, stiffen, plate, shear]","wang,t.k.","j.app.mech. 3,1947, a269.","[buckle, transverse, stiffen, plate, shear, pa..."


## 2. Ejercicio 2
### Puntuación máxima de la tarea: 2 puntos
#### Construir al menos dos modelos de recuperación de información basado en el modelo vectorial, visto en detalle en clases, cada uno con una configuración distinta de textos, tal y como se indica a continuación: solo title y title+text.

## 3. Ejercicio 3
### Puntuación máxima de la tarea: 5 puntos
#### Evaluar los modelos de recuperación de información construidos, mediante la  utilización de la colección de referencia dada (Archivos cran.qry.xml y cranqrel.trec.txt  ).  Seleccione dos métricas ampliamente utilizadas en la evaluación de este tipo de sistemas y analice los resultados para determinar cual es el mejor de los modelos. 